In [3]:
MY_INITIALS = "SF"

!pip -q install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

import pandas as pd
from PIL import Image
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import random
import numpy as np
from datetime import datetime
ranges = pd.read_csv("data/ranges_nocorr.csv")["0"]

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
guesses = []
truths = []
n_guess = 1
samples = random.sample(range(0, len(ranges)), 100)

idx = samples[n_guess - 1]
img = Image.open("data/newimgs/" + str(idx) + ".jpg", )

slider = widgets.FloatSlider(
         value=1,
         min=0,
         max=1,
         step=0.05,)
out = widgets.Output()

button = widgets.Button(description='Confirm')
nextb = widgets.Button(description="Next")

def on_conf(_):
    button.disabled = True
    nextb.disabled = False
    with out:
        global idx
        global n_guess
        guesses.append(slider.value)
        q = ranges[idx]
        truths.append(q)
        print(f"Correct answer was {q:.2f}, you were wrong by", round(abs(slider.value-q), 2))
        print("guess number", n_guess, "/100")
        if n_guess == 100:
            clear_output()
            results = pd.DataFrame(data = np.array([np.array(guesses), np.array(truths)]).T, columns = ["guesses", "truths"])
            now = datetime.now()
            path = "results" + now.strftime("%d_%m__%H_%M") + MY_INITIALS + ".csv"
            results.to_csv(path)      
            print("your mean error is", np.mean(np.abs(np.array(guesses) - np.array(truths))))
        n_guess = n_guess + 1

def on_next(_):
    button.disabled = False
    nextb.disabled = True
    with out:
        global idx
        clear_output()
        try:
            idx = samples[n_guess - 1]
            img = Image.open("data/newimgs/" + str(idx) + ".jpg")
            display(img)
        except:
            results = pd.DataFrame(data = np.array([np.array(guesses), np.array(truths)]).T, columns = ["guesses", "truths"])
            path = "results" + now.strftime("%d_%m__%H_%M") + MY_INITIALS + ".csv"
            results.to_csv(path)    
        
button.on_click(on_conf)
nextb.on_click(on_next)


box = widgets.VBox([slider, button, nextb, out])
with out:
    display(img)
box


